In [48]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pdb
G = nx.Graph()
G.add_edge(1,2)
G.add_edge(2,3)
G.add_edge(1,3)

G.node[1]['label']=1
G.node[2]['label']=2
G.node[3]['label']=3

G.edge[1][2]['weight']= 2.324

pos = nx.spring_layout(G)

nx.draw(G, pos)
node_labels = nx.get_node_attributes(G,'state')
nx.draw_networkx_labels(G, pos, labels = node_labels)
edge_labels = nx.get_edge_attributes(G,'state')
nx.draw_networkx_edge_labels(G, pos, labels = edge_labels)

plt.show()

SyntaxError: invalid syntax (<ipython-input-48-ddfe44041290>, line 15)